In [13]:
import datetime
from pathlib import Path
import atomic as atom
import definitions as defs
import visuals as vis 
import pandas as pd

from ta.volatility import BollingerBands
from ta.trend import MACD
from ta.momentum import RSIIndicator
from ta.trend import IchimokuIndicator
from ta.trend import EMAIndicator
from ta.trend import SMAIndicator
from ta.trend import ADXIndicator
from ta.volatility import KeltnerChannel

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [8]:
def getOPData(date, EnterTime):
    Banknifty_Path = "../NIFTYOptionsData/OptionsData/Banknifty/"
    Nifty_Path = "../NIFTYOptionsData/OptionsData/Nifty/"
    date_string = date.strftime("%Y/Data%Y%m%d.csv")
    BNPath = Banknifty_Path + date_string
    NPath = Nifty_Path + date_string
    my_fileN = Path(NPath)
    my_fileBN = Path(BNPath)
    # print("Working on file - "+date_string)
    if my_fileN.exists() and my_fileBN.exists():
      masterdfN = atom.LoadDF(NPath)
      masterdfBN = atom.LoadDF(BNPath)
    masterdf = masterdfBN

    symbol = defs.BN
    spotdata = atom.GetSpotData(masterdf, defs.BN)
    for s in range(len(spotdata)):
        currentcandle = spotdata.iloc[s]
        if currentcandle.name.time() == EnterTime:
            exp = atom.GetExpiry(masterdf, symbol)
            if symbol == defs.N :
                cst = currentcandle["open"]
                cst = int(round(cst / 50, 0)*50)
            elif symbol == defs.BN :
                cst = currentcandle["open"]
                cst = int(round(cst / 100, 0) * 100)
            opdfCE = masterdf[masterdf['symbol'] == symbol + exp + str(cst) + "CE"]
            opdfPE = masterdf[masterdf['symbol'] == symbol + exp + str(cst) + "PE"]
            break
    return (opdfCE, opdfPE)


In [37]:
# Conditions with RSI
def LongEnterCondition(s, opdata, longRSIThresh, ADXThresh):
    if (opdata["rsi"][s] < longRSIThresh) and (opdata["rsi"][s] > opdata["rsi"][s-1]) and (opdata["diplus"][s] > opdata["diminus"][s]):
        return True
    else:
        return False
    
def ShortEnterCondition(s, opdata, shortRSIThresh, ADXThresh):
    if (opdata["rsi"][s] > shortRSIThresh) and (opdata["rsi"][s] < opdata["rsi"][s-1]) and (opdata["diminus"][s] > opdata["diplus"][s]):# and (stock["adx"][s] > ADXThresh):
        return True
    else:
        return False
    
def LongExitCondition(s, opdata, longposition, longRSIThresh, SLPc):
   # Exit Condition based on longexit
    if (opdata["rsi"][s] > longRSIThresh) or (opdata["diplus"][s] < opdata["diminus"][s]):
        return True
    elif (opdata["close"][s] - longposition["Buy Price"]) < -1*SLPc*opdata["close"][s]:
        return True
    else:
        return False

def ShortExitCondition(s, opdata, shortposition, shortRSIThresh, SLPc):
   # Exit Condition based on longexit
    if (opdata["rsi"][s] < shortRSIThresh) or (opdata["diminus"][s] < opdata["diplus"][s]):
        return True
    elif (shortposition["Sell Price"] - opdata["close"][s]) < -1*SLPc*opdata["close"][s]:
        return True
    else:
        return False


In [40]:
# Conditions only with DI
def LongEnterCondition(s, opdata, longRSIThresh, ADXThresh):
    if (opdata["diplus"][s] > opdata["diminus"][s]):
        return True
    else:
        return False
    
def ShortEnterCondition(s, opdata, shortRSIThresh, ADXThresh):
    if (opdata["diminus"][s] > opdata["diplus"][s]):# and (stock["adx"][s] > ADXThresh):
        return True
    else:
        return False
    
def LongExitCondition(s, opdata, longposition, longRSIThresh, SLPc):
   # Exit Condition based on longexit
    if (opdata["diplus"][s] < opdata["diminus"][s]):
        return True
    elif (opdata["close"][s] - longposition["Buy Price"]) < -1*SLPc*opdata["close"][s]:
        return True
    else:
        return False

def ShortExitCondition(s, opdata, shortposition, shortRSIThresh, SLPc):
   # Exit Condition based on longexit
    if (opdata["diminus"][s] < opdata["diplus"][s]):
        return True
    elif (shortposition["Sell Price"] - opdata["close"][s]) < -1*SLPc*opdata["close"][s]:
        return True
    else:
        return False


In [38]:
longRSIThresh = 70
ADXThresh = 30
shortRSIThresh = 40
longpositions = []
longposition = {}
shortpositions = []
shortposition = {}
SLPc = 0.1
win = 20
lotsize = 25

date = datetime.date(2022, 1, 4)
EnterTime = datetime.time(9,16)
(opdfCE, opdfPE) = getOPData(date, EnterTime)
count = 0
LongActive = False
ShortActive = False
TotalPNL = 0

Rsi = RSIIndicator(opdfCE['close'], window=win)
opdfCE["rsi"] = Rsi.rsi()
Adx = ADXIndicator(opdfCE['high'], opdfCE['low'], opdfCE['close'], window = win)
opdfCE["adx"] = Adx.adx()
opdfCE["diplus"] = Adx.adx_pos()
opdfCE["diminus"] = Adx.adx_neg()
opdfCE = opdfCE.dropna()

for s in range(len(opdfCE)):
    count = count + 1
    if (count < len(opdfCE)) and (s > 0):
        if (LongEnterCondition(s, opdfCE, longRSIThresh, ADXThresh)) and not LongActive:
            print("Buying Long")
            longposition["Buy Time"] = opdfCE["time"][s+1]
            longposition["Buy Price"] = opdfCE["open"][s+1]
            LongActive = True
        if LongActive and LongExitCondition(s, opdfCE, longposition, longRSIThresh, SLPc):
            print("Selling Long")
            longposition["Sell Time"] = opdfCE["time"][s+1]
            longposition["Sell Price"] = opdfCE["open"][s+1]
            longposition["PNL"] = longposition["Sell Price"] - longposition["Buy Price"]
            TotalPNL = TotalPNL + longposition["PNL"]
            longpositions.append(longposition)
            LongActive = False
            longposition = {}
longpos = pd.DataFrame(longpositions)
print(longpos)

print("Total PNL in " + str(TotalPNL*lotsize))

Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
   Buy Time  Buy Price Sell Time  Sell Price     PNL
0  09:46:00     332.90  10:44:00      458.85  125.95
1  11:41:00     419.20  12:01:00      410.40   -8.80
2  12:04:00     436.70  12:13:00      432.15   -4.55
3  12:15:00     448.45  12:16:00      421.65  -26.80
4  12:43:00     391.10  12:46:00      375.35  -15.75
5  14:00:00     324.95  15:06:00      441.95  117.00
Total PNL in 4676.25


/Users/rishabhiyer/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/Users/rishabhiyer/opt/anaconda3/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/rishabhiyer/opt/anaconda3/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


In [41]:
longRSIThresh = 70
ADXThresh = 30
shortRSIThresh = 40
longpositions = []
longposition = {}
shortpositions = []
shortposition = {}
SLPc = 0.1
win = 20
lotsize = 25

date = datetime.date(2022, 1, 4)
EnterTime = datetime.time(9,16)
(opdfCE, opdfPE) = getOPData(date, EnterTime)
count = 0
LongActive = False
ShortActive = False
TotalPNL = 0

Rsi = RSIIndicator(opdfPE['close'], window=win)
opdfPE["rsi"] = Rsi.rsi()
Adx = ADXIndicator(opdfPE['high'], opdfPE['low'], opdfPE['close'], window = win)
opdfPE["adx"] = Adx.adx()
opdfPE["diplus"] = Adx.adx_pos()
opdfPE["diminus"] = Adx.adx_neg()
opdfPE = opdfPE.dropna()

for s in range(len(opdfPE)):
    count = count + 1
    if (count < len(opdfPE)) and (s > 0):
        if (LongEnterCondition(s, opdfPE, longRSIThresh, ADXThresh)) and not LongActive:
            print("Buying Long")
            longposition["Buy Time"] = opdfPE["time"][s+1]
            longposition["Buy Price"] = opdfPE["open"][s+1]
            LongActive = True
        if LongActive and LongExitCondition(s, opdfPE, longposition, longRSIThresh, SLPc):
            print("Selling Long")
            longposition["Sell Time"] = opdfPE["time"][s+1]
            longposition["Sell Price"] = opdfPE["open"][s+1]
            longposition["PNL"] = longposition["Sell Price"] - longposition["Buy Price"]
            TotalPNL = TotalPNL + longposition["PNL"]
            longpositions.append(longposition)
            LongActive = False
            longposition = {}
longpos = pd.DataFrame(longpositions)
shortpos = pd.DataFrame(shortpositions)
print(longpos)
print(shortpos)

print("Total PNL in " + str(TotalPNL*lotsize))

Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
    Buy Time  Buy Price Sell Time  Sell Price    PNL
0   09:37:00     249.30  09:46:00      227.20 -22.10
1   10:45:00     158.75  10:52:00      156.65  -2.10
2   11:06:00     157.90  11:08:00      152.05  -5.85
3   11:13:00     165.90  11:40:00      160.60  -5.30
4   12:03:00     160.85  12:04:00      148.85 -12.00
5   12:17:00     159.70  12:34:00      160.85   1.15
6   12:38:00     174.05  12:43:00      168.20  -5.85
7   12:47:00     180.70  12:49:00      171.50  -9.20
8   12:52:00     180.20  12:55:00      177.30  -2.90
9   13:02:00     183.65  13:03:00      166.00 -17.65
10  13:14:00     176.40  13:31:00      178.75   2.35
11  13:33:00     191.35  13:56:00      

/Users/rishabhiyer/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/Users/rishabhiyer/opt/anaconda3/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/rishabhiyer/opt/anaconda3/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
